# Real Estate Sales 2001-2016

The Office of Policy and Management maintains a listing of all real estate sales with a sales price of $2,000 or greater that occur between October 1 and September 30 of each year. For each sale record, the file includes: town, property address, date of sale, property type (residential, apartment, commercial, industrial or vacant land), sales price, and property assessment.

Source: <https://data.ct.gov/Housing-and-Development/Real-Estate-Sales-2001-2016/5mzw-sjtu>

**INPUT**
Each row is an individual sale that includes address, assessed value and sale price

**OUTPUT**
The processed dataset should contain aggregated counts per town per price bucket (<100k, <200k, <300k, <400k, 400k+)

**NOTES** We only care about Single Family homes, without ay Non-use codes applied to them

In [135]:
import pandas as pd
import csv

In [136]:
raw = pd.read_csv('../raw/Real_Estate_Sales_2001-2016.csv', index_col=0, dtype=str)

In [137]:
town2fips = pd.read_csv('https://raw.githubusercontent.com/CT-Data-Collaborative/ct-town-county-fips-list/master/ct-town-county-fips-list.csv', dtype=str, index_col=0)

### Extract only relevant sales from the dataset

In [147]:
# Single Family homes with Non-Use code = 0
relevant = raw.copy(deep=True)
relevant['NonUseCode'] = relevant['NonUseCode'].fillna(0).astype(str)

relevant = relevant[
    (relevant['ResidentialType'] == 'Single Family')
    & (relevant['NonUseCode'] == '0' )
]

# Sale Amount should be a positive float
relevant['SaleAmount'] = relevant['SaleAmount'].astype(float)
relevant = relevant[relevant['SaleAmount'] > 0]

# Remove all unnecessary columns
relevant = relevant.filter(['ListYear', 'Town', 'SaleAmount'])

relevant['Price Range'] = pd.cut(
    relevant['SaleAmount'],
    [0, 100000, 200000, 300000, 400000, 10000000000],
    labels=[ 'Less than $100,000',
             '$100,000 to $199,999',
             '$200,000 to $299,999',
             '$300,000 to $399,999',
             '$400,000 and Over']
)

In [148]:
relevant_counts = relevant.groupby(['Town', 'ListYear', 'Price Range']).size()
relevant_counts = relevant_counts.unstack(level=0,fill_value=0).stack().reset_index()

# Calculate Counties counts and append to the main dataframe
relevant_counts['County'] = relevant_counts['Town'].apply(lambda t: town2fips.loc[t]['County'])

counties = relevant_counts.groupby(['County', 'ListYear', 'Price Range']).sum()
counties = counties.unstack(level=0,fill_value=0).stack().reset_index()
counties['Town'] = counties['County'].apply(lambda c: c + ' County')

# Calculate CT values
ct = relevant_counts.groupby(['ListYear', 'Price Range']).sum()
#print(relevant_)
ct = ct.unstack(level=0,fill_value=0).stack().reset_index()
ct['Town'] = 'Connecticut'

relevant_counts_combined = pd.concat([relevant_counts, counties, ct], sort=False)

# Add remaining columns
relevant_counts_combined['FIPS'] = relevant_counts_combined['Town'].apply(lambda t: town2fips.loc[t]['FIPS'])
relevant_counts_combined['Measure Type'] = 'Number'
relevant_counts_combined['Variable'] = 'Number of Home Sales'

relevant_counts_combined.to_csv('../data/single-family-home-sales-2001-2016.csv', index=False,
                       columns=['Town', 'FIPS', 'ListYear', 'Price Range', 'Measure Type', 'Variable', 0],
                       header=['Town/County', 'FIPS', 'Year', 'Price Range', 'Measure Type', 'Variable', 'Value'],
                       quoting=csv.QUOTE_NONNUMERIC)

In [142]:
relevantst

,ListYear,Town,SaleAmount,Price Range
ID,,,,
6,2002,Andover,189900.0,"$100,000 to $199,999"
7,2002,Andover,189900.0,"$100,000 to $199,999"
13,2004,Andover,184000.0,"$100,000 to $199,999"
16,2003,Andover,218000.0,"$200,000 to $299,999"
19,2004,Andover,279900.0,"$200,000 to $299,999"
21,2003,Andover,220000.0,"$200,000 to $299,999"
23,2002,Andover,146000.0,"$100,000 to $199,999"
24,2002,Andover,146000.0,"$100,000 to $199,999"
34,2002,Andover,129000.0,"$100,000 to $199,999"
